In [59]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
from nltk.stem import WordNetLemmatizer #lemitization
from nltk.corpus import words
from tqdm import tqdm
tqdm.pandas()
import seaborn as sns
import matplotlib.pyplot as plt

from math import log2

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aless\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aless\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\aless\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [60]:
data = pd.read_csv('../../data/processed/processed_xy_augmented.csv')
change_in_pop = data['test_mean_clicks'].div(data['test_mean_impressions'])-data['clicks'].div(data['impressions'])


In [61]:
def emotional_score(data,emotion,length):
    lengths = data[length].apply(lambda x: log2(x+1))
    return data[emotion].div(lengths)


In [62]:
print(emotional_score(data,'Negative','token_word_count'))

0        1.477829
1        1.477829
2        1.477829
3        0.000000
4        0.000000
           ...   
29797    0.000000
29798    0.000000
29799    0.000000
29800    0.000000
29801    0.000000
Length: 29802, dtype: float64


In [63]:
def vizualize(data,emotion,length,y):
    escore= emotional_score(data,emotion,length)
    sns.regplot(x=escore,y=y)
    plt.savefig(f'../../data/visualizations/{emotion}-{length}.png')
    plt.close()
    

In [64]:
for emotion in ['Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']:
    for length in ['token_word_count','emotive_word_count']:
        vizualize(data,emotion,length,change_in_pop)